In [5]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix

In [6]:
# Skip date
df1 = pd.read_csv('combined_data_1.txt', header = None, names = ['User_Id', 'Rating','Date'])

df1['Rating'] = df1['Rating'].astype(float)

print('Dataset 1 shape: {}'.format(df1.shape))
print('-Dataset examples-')
print(df1.iloc[::5000000, :])#START:STOP:STEP (5000000)

Dataset 1 shape: (24058263, 3)
-Dataset examples-
          User_Id  Rating        Date
0              1:     NaN         NaN
5000000   2560324     4.0  2005-12-06
10000000  2271935     2.0  2005-04-11
15000000  1921803     2.0  2005-01-31
20000000  1933327     3.0  2004-11-10


In [7]:
df1.index = np.arange(0,len(df1))
# get movie count (where rating ==NaN)
movie_count = df1.isnull().sum()[1] #df.isnull() searches for NaN values across all columns, we get in [1] column-rating

# get customer count (amount of unique user_id-movie_id)
user_count = df1['User_Id'].nunique() - movie_count

# get rating count (for ratings not only unique, all values except movies ids)
rating_count = df1['User_Id'].count() - movie_count

movie_count, user_count , rating_count

(4499, 470758, 24053764)

In [25]:
#Movie ID is really a mess import! 
#Looping through dataframe to add Movie ID column WILL make the Kernel run out of memory as it is too inefficient. 
#I achieve my task by first creating a numpy array with correct length 
#then add the whole array as column into the main dataframe!

#WORKING APPROXIMATELY 5 MIN

df_nan = pd.DataFrame(pd.isnull(df1.Rating)) #new df with indexes standard and one column "Rating" with True/False values
#df1.Rating gets indexes+"Rating" column values
#pd.isnull(df1.Rating) gets indexes+"Rating" mask column values (True=NaN, False-otherwise)

df_nan = df_nan[df_nan['Rating'] == True]#new df with old indexes and one column "Rating" with only True values
df_nan = df_nan.reset_index() # adds new indexes, old-moves to new column "index"


 # zip(df_nan['index'][1:],df_nan['index'][:-1]) returns iterator of tuples

movie_np = []
movie_id = 1 #we can start with 1 and sum it in loop,because all movie_id sorted in dataset from 1 to last (increase order)

# i-current movie_id j-previous movie_id , its needed for counting the difference because
#we need to know amount of rows=amount of ratings 
# so (i-j-1)=amount of not null ratings of whole users dataset for one movie (which has movie_id)
for i,j in zip(df_nan['index'][1:],df_nan['index'][:-1]):
    # numpy approach
    temp = np.full((1,i-j-1), movie_id) # 1st arg-shape, second-value add row , because movie_np is array 
    temp
    movie_np = np.append(movie_np, temp) # adds element to the end of array
    movie_id += 1 #we can sum it in loop дшлу +1 ,because all movie_id sorted in dataset from 1 to last (increase order)
    
# Account for last record and corresponding length
# numpy approach
last_record = np.full((1,len(df1) - df_nan.iloc[-1, 0] - 1),movie_id)
movie_np = np.append(movie_np, last_record)
#finally after loop we get array movie_np size=amount of non null ratings, at each index it has corresponding 
#movie_id value for tis rating +can be concatenated with cleaned df
print('Length: {}'.format(len(movie_np)))

Length: 24053764


In [27]:
# remove those Movie ID rows from initial df (rows where rating=NaN)
df1 = df1[pd.notnull(df1['Rating'])]

df1['Movie_Id'] = movie_np.astype(int) #Add movie_id column in df as movie_np received on previous step
df1['User_Id'] = df1['User_Id'].astype(int)
print('-Dataset examples-')
print(df1.iloc[::5000000, :])

-Dataset examples-
          User_Id  Rating        Date  Movie_Id
1         1488844     3.0  2005-09-06         1
5000996    501954     2.0  2004-08-26       996
10001962   404654     5.0  2005-08-29      1962
15002876   886608     2.0  2005-09-19      2876
20003825  1193835     2.0  2003-08-13      3825
